In [1]:
import pandas as pd
import numpy as np
import sys
import pickle
import warnings
warnings.filterwarnings("ignore")

sys.path.insert(0, '..')
from DataModule.Data_Preparation import CoronnaCERTAINDataset
import FineTuneModule
import EvaluationModule

import xgboost as xgb
import sklearn
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import ShuffleSplit,cross_val_score
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.pipeline import Pipeline

import submitit

(CVXPY) Jul 12 11:52:06 PM: Encountered unexpected exception importing solver CVXOPT:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 12 11:52:06 PM: Encountered unexpected exception importing solver GLPK:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 12 11:52:06 PM: Encountered unexpected exception importing solver GLPK_MI:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/e

In [2]:
def main():
    remove_low_DAS_list = [True, False]
#     remove_low_DAS_list = [True]
#     challenge_list = ["regression", "regression_delta", "classification", "binary_classification"]
    challenge_list = ["regression_delta_binary"]
    imputation_list = ["SimpleFill", "KNN", "SoftImpute", "IterativeImputer"]
#     imputation_list = ["KNN"]
    process_approach_list = ['KVB','SC']
#     process_approach_list = ['SC']
    patient_group_list = ["all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"]
#     patient_group_list = ["bionaive TNF"]
    for remove_low_DAS in remove_low_DAS_list:
        for challenge in challenge_list:
            for imputation in imputation_list:
                for process_approach in process_approach_list:
                    for patient_group in patient_group_list:
                        print("remove_low_DAS",remove_low_DAS)
                        print("clallenge:",challenge)
                        print("imputation:", imputation)
                        print("process_approach", process_approach)
                        print("patient_group", patient_group)
                        dataset = CoronnaCERTAINDataset(
                            library_root = '/Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/',
                            challenge = challenge, #option: regression, regression_delta, classification, binary_classification
                            dataset = 'CORRONA CERTAIN', 
                            process_approach = process_approach, #option: KVB, SC
                            imputation = imputation, #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
                            patient_group =patient_group, #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
                            drug_group = 'all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
                            time_points = (0,3), 
                            train_test_rate = 0.8,
                            remove_low_DAS = remove_low_DAS,
                            save_csv = False, 
                            random_state = 2022,
                            verbose=False)
                        # read train, test from dataloader
                        train_set, train_loc = dataset.get_train()
                        test_set, test_loc = dataset.get_test()
                        
                        aml = EvaluationModule.AutoBuild(seed=dataset.random_state, project_name="SC_Jul12", challenge=dataset.challenge)
                        # define models
                        if "regression" in dataset.challenge:
                            model = RandomForestRegressor()
                        elif "classification" in dataset.challenge:
                            model = FineTuneModule.fine_tune(train=train_set, model="drf_classification", search_methods="RandomSearch")
#                             model = RandomForestClassifier()
                        aml.validate("rf", model, train_set, test_set)
                        aml.validation_output(dataset)
                        aml.test_output(dataset)
                        print("---------------------------------------------------")

In [3]:
main()

remove_low_DAS True
clallenge: regression_delta_binary
imputation: SimpleFill
process_approach KVB
patient_group all
---------------------------------------------------
remove_low_DAS True
clallenge: regression_delta_binary
imputation: SimpleFill
process_approach KVB
patient_group bioexp nTNF
---------------------------------------------------
remove_low_DAS True
clallenge: regression_delta_binary
imputation: SimpleFill
process_approach KVB
patient_group bionaive TNF
---------------------------------------------------
remove_low_DAS True
clallenge: regression_delta_binary
imputation: SimpleFill
process_approach KVB
patient_group bionaive orencia
---------------------------------------------------
remove_low_DAS True
clallenge: regression_delta_binary
imputation: SimpleFill
process_approach KVB
patient_group KVB
---------------------------------------------------
remove_low_DAS True
clallenge: regression_delta_binary
imputation: SimpleFill
process_approach SC
patient_group all
---------